# Sprawozdanie - MOwNiT - laboratorium 4., zadanie 2.

#### Autor: Jakub Psarski

## Obraz binarny

Wygeneruj losowy obraz binarny o rozmiarze $n \times n$ i wybranej gęstości $δ$ czarnych punktów $δ = 0.1, 0.3, 0.4$. Korzystając z różnego typu sąsiedztwa (4-sąsiadów, 8-sąsiadów, 8-16-sąsiadów) zaproponuj funkcję energii (np. w bliskiej odległości te same kolory przyciągają się, a w dalszej odpychają się, funkcja energii może być również adaptacją modelu Isinga) i dokonaj jej minimalizacji za pomocą algorytmu symulowanego wyżarzania. W jaki sposób można generować stany sąsiednie? Jak różnią się uzyskane wyniki w zależności od rodzaju sąsiedztwa, wybranej funkcji energii i szybkości spadku temperatury?